# Using enterprise to analyze PTA data

In this notebook you will learn:
* How to use `enterprise` to interact with IPTA data,
* How to setup an analysis of indiviudual pulsar noise properties,
* How to search in PTA data for GWs,
* How to perform Bayesian model selection,
* How to post-process your results.

# Pre-requisites (installation etc.)

- **Install `miniconda` locally**

    `wget -q https://repo.continuum.io/miniconda/Miniconda2-latest-Linux-x86_64.sh`
    
    `bash Miniconda2-latest-Linux-x86_64.sh -b -p ~/.local/opt/miniconda2`
    
    `rm Miniconda2-latest-Linux-x86_64.sh`


- **Add miniconda’s `python`  to the front of your `$PATH`**

    `echo "export PATH=$HOME/.local/opt/miniconda2/bin:$PATH" >> .bashrc`
    
    `source .bashrc`


- **Install the basic python packages**

    `conda install -y numpy==1.13.3 cython scipy`


- **Install latest `libstempo` from GitHub with `pip`.  `tempo2` should be installed automatically.  Add extra ephemerides if needed**

    `pip install git+https://github.com/vallis/libstempo@master`


- **Install more python packages**

    `conda install -y matplotlib ipython h5py mpi4py numexpr statsmodels astropy ephem`


- **Install non-conda packages with `pip`**

    `pip install healpy acor line_profiler jplephem corner numdifftools`


- [optional] **Install `scikit-sparse`**

    `conda install -c menpo scikit-sparse`
   
   
- [optional] **Alternatively install `suite sparse` and then use pip to install `scikit-sparse` (maybe needed with python 3.6)**

    `conda install -c conda-forge suitesparse`
    
    `pip install git+https://github.com/scikit-sparse/scikit-sparse.git@master`


- **Install PTMCMC for sampling**

    `pip install git+https://github.com/jellis18/PTMCMCSampler@master`


- **Finally, install enterprise**

    `pip install git+https://github.com/nanograv/enterprise@master`

# Load modules

In [ ]:
% matplotlib inline
%config InlineBackend.figure_format = 'retina'
%load_ext autoreload
%autoreload 2

from __future__ import division

import numpy as np
import os, glob, json 
import matplotlib.pyplot as plt
import scipy.linalg as sl

import enterprise
from enterprise.pulsar import Pulsar
import enterprise.signals.parameter as parameter
from enterprise.signals import utils
from enterprise.signals import signal_base
from enterprise.signals import selections
from enterprise.signals.selections import Selection
from enterprise.signals import white_signals
from enterprise.signals import gp_signals
from enterprise.signals import deterministic_signals
import enterprise.constants as const

import corner
from PTMCMCSampler.PTMCMCSampler import PTSampler as ptmcmc

# Define a python dictionary of pulsar names and PTAs

In [ ]:
# The pulsars we'll be analyzing
psrdict = {'J1713+0747': [{'pta': ['NANOGrav', 'PPTA']}], 
           'J1909-3744': [{'pta': ['NANOGrav', 'PPTA']}], 
           'J1640+2224': [{'pta': ['NANOGrav']}], 
           'J1600-3053': [{'pta': ['NANOGrav']}],
           'J2317+1439': [{'pta': ['NANOGrav']}], 
           'J1918-0642': [{'pta': ['NANOGrav']}], 
           'J1614-2230': [{'pta': ['NANOGrav']}], 
           'J1744-1134': [{'pta': ['NANOGrav', 'PPTA']}],
           'J0030+0451': [{'pta': ['NANOGrav']}], 
           'J2145-0750': [{'pta': ['NANOGrav']}], 
           'J1857+0943': [{'pta': ['NANOGrav']}], 
           'J1853+1303': [{'pta': ['NANOGrav']}], 
           'J0613-0200': [{'pta': ['NANOGrav']}],
           'J1455-3330': [{'pta': ['NANOGrav']}], 
           'J1741+1351': [{'pta': ['NANOGrav']}], 
           'J2010-1323': [{'pta': ['NANOGrav']}], 
           'J1024-0719': [{'pta': ['NANOGrav']}], 
           'J1012+5307': [{'pta': ['NANOGrav']}],
           'J0437-4715': [{'pta': ['PPTA']}]
          }
psrlist=psrdict.keys()

In [ ]:
psrlist

## Get par, tim, and noise files
Here we collect the tim and par files as well as noise files made from the `PAL2` code. These are the same par, tim, and noise files used in the 9-year analysis papers. We use the convienience function above to convert from `PAL2` noise files to `enterprise` parameter dictionaries.

In [ ]:
datadir = './partim_filtered_ppta_ng/'

In [ ]:
parfiles = sorted(glob.glob(datadir + '/*.par'))
timfiles = sorted(glob.glob(datadir + '/*.tim'))

# filter
parfiles = [x for x in parfiles if x.split('/')[-1].split('.')[0] in psrlist]
timfiles = [x for x in timfiles if x.split('/')[-1].split('.')[0] in psrlist]

In [ ]:
len(parfiles)

## Load into Pulsar class list

* The `enterprise` Pulsar class uses `libstempo` to read in `par` and `tim` files, then stores all pulsar data into a `Pulsar` object. This object contains all data and meta-data needed for the ensuing pulsar and PTA analysis. You no longer to reference the `par` and `tim` files after this cell.
* Note below that you can explicitly declare which version of the JPL solar-system ephemeris model that will be used to compute the Roemer delay between the geocenter and the barycenter (e.g. `DE436`). Otherwise the default values will be taken from the `par` files. Explicitly declaring the version here is good practice.
* You can also explicitly set the clock file to a version of `BIPM`, e.g. `BIPM(2015)`. This is less important, and you can let the code take the value from the `par` file.
* When you execute the following cell, you will get warnings like `WARNING: Could not find pulsar distance for PSR ...`. Don't worry! This is expected, and fine. Not all pulsars have well constrained distances, and will be set to `1 kpc` with a `20%` uncertainty.

In [ ]:
psrs = []
for p, t in zip(parfiles[:2], timfiles[:2]):
    psr = Pulsar(p, t, ephem='DE436', clk=None)
    psrs.append(psr)

# PTA Parameter Estimation

* We can read-in some previously computed noise properties from single-pulsar analyses. These are things like `EFAC`, `EQUAD`, and (for `NANOGrav`) `ECORR`. 
* In practice, we set these white-noise properties as fixed in the low-frequency noise / GW searches.
* The noise properties have been stored as `json` files, and are read in to a big parameter dictionary.

In [ ]:
noisefiles = sorted(glob.glob('./partim_filtered_ppta_ng/noisefiles_ppta_ng_normal/*.json'))

In [ ]:
params = {}
for nf in noisefiles:
    with open(nf, 'r') as fin:
        params.update(json.load(fin))

## Set up `enterprise` model for PTA upper-limit (*verbose version*)

* Usually, in a full PTA analysis we fix all of the white noise (EFAC, EQUAD, and ECORR) parameters to the values obtained from the noise files. This is done by using `Constant` parameters. In this case we do not specify a default value for all instances of that parameter but instead will set them, based on their initialized pulsar and backend specific name, later via the `set_default_params` method of `PTA`. 

* We use the `Selection` object to define which noise parameters are assigned to which chunks of TOAs. This selection is based on unique combination of backends and receivers.

* Another feature to notice is that **for upper limits** we do not use a uniform prior on the log of the red-noise or GWB amplitude. Instead we use a `LinearExp` prior (short for linear-exponent prior), that is a prior of the form $p(x)\propto 10^x$. This is how we can still use the log of the parameter to sample but place a uniform prior on the parameter itself. We do this for both the red noise and GWB amplitude parameters. **For detection analyses** we still use a `Uniform` prior on the log of the red-noise and GWB amplitude. 

* In order to save on computing time we do not include spatial correlations here. Instead we model the GWB as a common red process across all pulsars. In `enterprise` we can do this with a simple trick. We pre-initialize the parameters before passing them to the `Signal` model. In this way the *same* parameter instance is used for all pulsars. Lastly, we fixt the spectral index of the GWB to be 13/3 (4.33) using the `Constant` parameter.

In [ ]:
# find the maximum time span to set GW frequency sampling
tmin = [p.toas.min() for p in psrs]
tmax = [p.toas.max() for p in psrs]
Tspan = np.max(tmax) - np.min(tmin)

In [ ]:
# define selection by observing backend
selection = selections.Selection(selections.by_backend)

# special selection for ECORR only use wideband NANOGrav data
selection2 = selections.Selection(selections.nanograv_backends)

### Parameters

In [ ]:
# white noise parameters
# since we are fixing these to values from the noise file we set
# them as constant parameters
efac = parameter.Constant()
equad = parameter.Constant()
ecorr = parameter.Constant()

# red noise parameters
log10_A = parameter.LinearExp(-20, -11)
gamma = parameter.Uniform(0, 7)

# dm-variation parameters
log10_A_dm = parameter.LinearExp(-20, -11)
gamma_dm = parameter.Uniform(0, 7)

# GW parameters (initialize with names here to use parameters in common across pulsars)
log10_A_gw = parameter.LinearExp(-18,-12)('log10_A_gw')
gamma_gw = parameter.Constant(4.33)('gamma_gw')

### Signals

In [ ]:
# white noise
ef = white_signals.MeasurementNoise(efac=efac, selection=selection)
eq = white_signals.EquadNoise(log10_equad=equad, selection=selection)
ec = white_signals.EcorrKernelNoise(log10_ecorr=ecorr, selection=selection2)

# red noise (powerlaw with 30 frequencies)
pl = utils.powerlaw(log10_A=log10_A, gamma=gamma)
rn = gp_signals.FourierBasisGP(spectrum=pl, components=30, Tspan=Tspan)

# DM-variations (powerlaw with 30 frequencies)
dm_basis = utils.createfourierdesignmatrix_dm(nmodes=30, Tspan=Tspan)
dm_pl = utils.powerlaw(log10_A=log10_A_dm, gamma=gamma_dm)
dm_gp = gp_signals.BasisGP(dm_pl, dm_basis, name='dm_gp')

# gwb (no spatial correlations)
cpl = utils.powerlaw(log10_A=log10_A_gw, gamma=gamma_gw)
gw = gp_signals.FourierBasisGP(spectrum=cpl, components=30, Tspan=Tspan, name='gw')

# for spatial correltions you can do...
#orf = utils.hd_orf()
#crn = gp_signals.FourierBasisCommonGP(cpl, orf, components=30, Tspan=Tspan, name='gw')

# to add solar system ephemeris modeling...
eph = deterministic_signals.PhysicalEphemerisSignal(use_epoch_toas=True)

# timing model
tm = gp_signals.TimingModel(use_svd=True)

In [ ]:
# full model
s = ef + eq + rn + dm_gp + tm + eph + gw

In [ ]:
# intialize PTA, adding ecorr only for NANOGrav pulsars
models = []
        
for p in psrs:    
    if 'NANOGrav' in p.flags['pta']:
        s2 = s + ec 
        models.append(s2(p))
    else:
        models.append(s(p))
    
pta = signal_base.PTA(models)

In [ ]:
pta.params

### Set white noise parameters

In [ ]:
pta.set_default_params(params)

### Set initial parameters drawn from prior

In [ ]:
x0 = np.hstack(p.sample() for p in pta.params)
ndim = len(x0)

### Set up sampler

In [ ]:
# initial jump covariance matrix
cov = np.diag(np.ones(ndim) * 0.01**2)

sampler = ptmcmc(ndim, pta.get_lnlikelihood, pta.get_lnprior, cov, 
                 outDir='./chains/ipta_dr2_ng_ppta_gwb/', resume=False)

### Sample!

In [ ]:
# sampler for N steps
N = int(5e6)
x0 = np.hstack(p.sample() for p in pta.params)
sampler.sample(x0, N, SCAMweight=30, AMweight=15, DEweight=50, )

### Plot output

In [ ]:
chain = np.loadtxt('./chains/ipta_dr2_ng_ppta_gwb/chain_1.txt')
burn = int(0.25 * chain.shape[0])

In [ ]:
plt.hist(chain[burn:,-5], 50, normed=True, histtype='step', lw=2);

### Upper limit value

In [ ]:
upper = 10**np.percentile(chain[burn:, -5], q=95)
print(upper)

## Add other custom functions to model

In [ ]:
## we add a new custom function that takes the form of a dispersive (1 / \nu**2) 
## exponential dip in the residuals of `J1713+0747` due to a void in the ISM 
## plasma. This is a real effect that has been observed.

@signal_base.function
def chrom_exp_decay(toas, freqs, log10_Amp=-7,
                    t0=54000, log10_tau=1.7, idx=2):
    """
    Chromatic exponential-dip delay term in TOAs.

    :param t0: time of exponential minimum [MJD]
    :param tau: 1/e time of exponential [s]
    :param log10_Amp: amplitude of dip
    :param idx: index of chromatic dependence

    :return wf: delay time-series [s]
    """
    t0 *= const.day
    tau = 10**log10_tau * const.day
    wf = -10**log10_Amp * np.heaviside(toas - t0, 1) * \
        np.exp(- (toas - t0) / tau)

    return wf * (1400 / freqs) ** idx

def dm_exponential_dip(tmin, tmax, idx=2, name='dmexp'):
    """
    Returns chromatic exponential dip (i.e. TOA advance):

    :param tmin, tmax:
        search window for exponential dip time.
    :param idx:
        index of radio frequency dependence (i.e. DM is 2). If this is set
        to 'vary' then the index will vary from 1 - 6
    :param name: Name of signal

    :return dmexp:
        chromatic exponential dip waveform.
    """
    t0_dmexp = parameter.Uniform(tmin,tmax)
    log10_Amp_dmexp = parameter.Uniform(-10, -2)
    log10_tau_dmexp = parameter.Uniform(np.log10(5), np.log10(100))
    wf = chrom_exp_decay(log10_Amp=log10_Amp_dmexp, t0=t0_dmexp,
                         log10_tau=log10_tau_dmexp, idx=idx)
    dmexp = deterministic_signals.Deterministic(wf, name=name)

    return dmexp

In [ ]:
## we add a new custom function that takes the form of a dispersive (1 / \nu**2) 
## annual term. This is a real effect that can be observed in NANOGrav DMX values.

@signal_base.function
def chrom_yearly_sinusoid(toas, freqs, log10_Amp=-7, phase=0, idx=2):
    """
    Chromatic annual sinusoid.

    :param log10_Amp: amplitude of sinusoid
    :param phase: initial phase of sinusoid
    :param idx: index of chromatic dependence

    :return wf: delay time-series [s]
    """

    wf = 10**log10_Amp * np.sin( 2 * np.pi * const.fyr * toas + phase)
    return wf * (1400 / freqs) ** idx

def dm_annual_signal(idx=2, name='dm_s1yr'):
    """
    Returns chromatic annual signal (i.e. TOA advance):

    :param idx:
        index of radio frequency dependence (i.e. DM is 2). If this is set
        to 'vary' then the index will vary from 1 - 6
    :param name: Name of signal

    :return dm1yr:
        chromatic annual waveform.
    """
    log10_Amp_dm1yr = parameter.Uniform(-10, -2)
    phase_dm1yr = parameter.Uniform(0, 2*np.pi)

    wf = chrom_yearly_sinusoid(log10_Amp=log10_Amp_dm1yr,
                               phase=phase_dm1yr, idx=idx)
    dm1yr = deterministic_signals.Deterministic(wf, name=name)

    return dm1yr

In [ ]:
# full model
s = ef + eq + rn + dm_gp + tm + eph + gw

# add annual DM term for all pulsars
s += dm_annual_signal()

In [ ]:
# add a DM exponential dip for J1713+0747
models = []

for p in psrs:    
    if 'NANOGrav' in p.flags['pta']:
        s2 = s + ec 
        if p.name == 'J1713+0747':
            s3 = s2 + dm_exponential_dip(tmin=(np.min(tmin)/86400.0), 
                                         tmax=(np.max(tmax)/86400.0))
            models.append(s3(p))
        else:
            models.append(s2(p))
    else:
        models.append(s(p))
        
pta = signal_base.PTA(models)

In [ ]:
pta.set_default_params(params)

In [ ]:
x0 = np.hstack(p.sample() for p in pta.params)
ndim = len(x0)

In [ ]:
pta.params

In [ ]:
# initial jump covariance matrix
cov = np.diag(np.ones(ndim) * 0.01**2)

sampler = ptmcmc(ndim, pta.get_lnlikelihood, pta.get_lnprior, cov, 
                 outDir='./chains/ipta_dr2_ng_ppta_gwb/', resume=False)

In [ ]:
# sampler for N steps
N = int(5e6)
x0 = np.hstack(p.sample() for p in pta.params)
sampler.sample(x0, N, SCAMweight=30, AMweight=15, DEweight=50, )

## Now, the easy way to do all of this...

We use `enterprise_extensions` as in the single-pulsar analysis tutorial.

In [ ]:
import enterprise_extensions
from enterprise_extensions import models, model_utils

In [ ]:
## Note that we all still have work to do! This function can do everything
## as the routine above, but will not add the DM exponential dip in J1713+0747.

pta = models.model_general(psrs, psd='powerlaw', noisedict=params, components=30, 
                      gamma_common=4.33, upper_limit=True, bayesephem=True, 
                      dm_var=True, dm_type='gp', dm_psd='powerlaw', dm_annual=False)

In [ ]:
# Setup an instance of a HyperModel.
# This class currently works with pulsars having unique noise model
# descriptions for custom proposal distributoons and jumps.
super_model = model_utils.HyperModel({0: pta})

In [ ]:
outdir = './chains/ipta_dr2_ng_ppta_gwb/'
sampler = super_model.setup_sampler(resume=False, outdir=outdir)

In [ ]:
# sampler for N steps
N = int(5e6)
x0 = super_model.initial_sample()

In [ ]:
# sample
sampler.sample(x0, N, SCAMweight=30, AMweight=15, DEweight=50, )

In [ ]:
# Read in chains and parameters

chain = np.loadtxt(outdir + '/chain_1.txt')
burn = int(0.25*chain.shape[0])
pars = np.loadtxt(outdir + '/pars.txt', dtype=np.unicode_)

pp = model_utils.PostProcessing(chain, pars)

In [ ]:
# Plot GW amplitude posterior
ind = list(pars).index('log10_A_gw')
plt.hist(chain[burn:,ind], bins=40);

In [ ]:
# Compute upper limit
print model_utils.ul(chain[burn:, ind], q=95.0)

# PTA Model Selection

We want to be able to compute the Bayesian odds for a GWB in the data. This can be done using the same model selection scheme as in the single-pulsar noise analysis, where we now choose between a model with a common (but uncorrelated) red process in the pulsars, and a GWB affecting all pulsars.

We typically perform detection-type analyses with uniform-in-log priors on all amplitude parameters for low-frequency processes. This is implemented below whenever we switch `upper_limit` to be equal to `False`.

## Setup dictionary of PTA models

In [ ]:
nmodels = 2
mod_index = np.arange(nmodels)

# Make dictionary of PTAs.
pta = dict.fromkeys(mod_index)
pta[0] = models.model_general(psrs, psd='powerlaw', noisedict=params, components=30, 
                      gamma_common=4.33, upper_limit=False, bayesephem=True, 
                      dm_var=True, dm_type='gp', dm_psd='powerlaw', dm_annual=False)
pta[1] = models.model_general(psrs, psd='powerlaw', noisedict=params, orf='hd', components=30, 
                      gamma_common=4.33, upper_limit=False, bayesephem=True, 
                      dm_var=True, dm_type='gp', dm_psd='powerlaw', dm_annual=False)

In [ ]:
super_model = model_utils.HyperModel(pta)

## Sample

In [ ]:
sampler = super_model.setup_sampler(resume=False, outdir=outdir)

In [ ]:
# sampler for N steps
N = int(5e6)
x0 = super_model.initial_sample()

In [ ]:
# sample
sampler.sample(x0, N, SCAMweight=30, AMweight=15, DEweight=50, )

## Post-process

In [ ]:
chain = np.loadtxt(outdir + '/chain_1.txt')
burn = int(0.25*chain.shape[0])
pars = np.loadtxt(outdir + '/pars.txt', dtype=np.unicode_)

pp = model_utils.PostProcessing(chain, pars)

In [ ]:
# Plot histgram for GW amplitude
chain_burn = chain[burn:,:]

ind_model = list(pars).index('nmodel')
ind_gwamp = list(pars).index('log10_A_gw')

# ORF = None
#plt.hist(chain_burn[chain_burn[:, ind_model] < 0.5, ind_gwamp], bins=40);

# ORF = Hellings & Downs
plt.hist(chain_burn[chain_burn[:, ind_model] > 0.5, ind_gwamp], bins=40);

In [ ]:
# Plot histogram for GWB model selection
plt.hist(chain_burn[:, ind_model], bins=40);

## Bayes factors

### Savage-Dickey Bayes factor

This gives the signal-vs-noise Bayes factor for a common red process in the pulsars plus intrisnc noise, versus intrinsic noise alone.

In [ ]:
print model_utils.bayes_fac(chain_burn[chain_burn[:, ind_model] < 0.5, ind_gwamp], ntol=1)

### Posterior odds ratio

This gives the Bayesian odds between a model with a Hellings & Downs correlated red process between pulsars, and a common (but uncorrelated) red process between pulsars. This is the smoking-gun detection statsitic for a GWB signal.

In [ ]:
print model_utils.odds_ratio(chain_burn[:, ind_model], models=[0,1])